In [1]:
import time 
import datetime
import warnings
import pandas as pd
import numpy as np
from numpy import absolute
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.width = 1000
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, MaxAbsScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold, GridSearchCV
from sklearn.metrics import precision_recall_fscore_support, mean_squared_error
import joblib
import xgboost as xgb
from xgboost import XGBRegressor
from datetime import datetime
from datetime import datetime


In [55]:
df = pd.read_csv("data.csv", parse_dates=["event_date"])
df["event_description"].fillna("", inplace=True)
df.head(5)

,Unnamed: 0.1,Unnamed: 0,event_date,event_title,event_description,location_description,location_accuracy,landslide_category,landslide_trigger,landslide_size,landslide_setting,fatality_count,injury_count,country_name,admin_division_name,admin_division_population,gazeteer_closest_point,gazeteer_distance,longitude,latitude,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipcover,windgust,windspeed,winddir,pressure,cloudcover,visibility,moonphase,conditions,stations,elevation,continent,season,treecover2000,loss,gain,soil_texture_0,soil_texture_10,soil_texture_30,soil_texture_60,soil_texture_100,soil_texture_200,population_density_2000,population_density_2005,population_density_2010,population_density_2015,population_density_2020
0,0,0,1988-11-07 00:00:00,Sta. Rosa del Sur,"He recalled that on Nov.7, 1988, 18 families w...",Sta. Rosa del Sur,25km,landslide,continuous_rain,medium,unknown,0.0,0.0,Philippines,Bicol,10531,Pasacao,1.16205,123.0319,13.5107,27.6,24.1,26.4,31.2,24.1,28.5,23.7,85.49,87.00,66.67,50.955865,50.4,161.7,1004.8,100.0,8.3,0.98,"Rain, Overcast",['98440099999'],42.0,Asia,autumn,97,0.0,0,Cl,Cl,Cl,Cl,Cl,Cl,542.317627,672.570496,801.507996,944.788940,1095.986572
1,1,1,1993-06-04 06:00:00,Holbeck Hall Hotel,The first signs of movement on the cliff were ...,Holbeck Hall Hotel,1km,other,rain,medium,deforested_slope,0.0,0.0,United Kingdom,England,38909,Scarborough,1.64675,-0.3913,54.2670,16.3,7.9,12.2,16.3,6.2,11.9,9.0,81.30,0.00,0.00,50.955865,17.5,163.0,1020.8,68.5,18.0,0.50,Partially cloudy,"['03292099999', '03384099999', '03257099999', ...",59.0,Europe,spring,0,0.0,0,SaLo,SaLo,SaLo,SaLo,SaLo,SaLo,1373.713623,1372.173340,1394.869263,1403.645020,1409.767212
2,2,2,1995-08-14 00:00:00,Caux : 50 000 m3 of debris flows,50 000 m3 of debris flows,"Vaud, Switzerland",10km,debris_flow,downpour,large,unknown,0.0,0.0,Switzerland,Vaud,2000,Caux,4.92572,6.9932,46.4093,25.1,13.1,19.5,25.1,13.1,19.5,13.5,71.05,4.37,4.17,50.955865,14.8,151.8,1016.4,26.2,12.0,0.59,"Rain, Partially cloudy","['06712099999', '06628099999', '06610099999', ...",1845.0,Europe,summer,66,0.0,0,Lo,Lo,Lo,Lo,Lo,Lo,130.772888,142.286316,158.012466,175.659775,191.833191
3,3,3,1996-12-26 00:00:00,Western Columbia River Gorge,The western part of the Columbia Gorge on the ...,Western Columbia River Gorge,50km,debris_flow,freeze_thaw,small,above_river,0.0,0.0,United States,Oregon,1144,Cascade Locks,19.22137,-122.0991,45.5775,1.7,-0.7,0.1,1.7,-7.8,-3.5,-1.3,90.30,24.90,66.67,50.955865,37.1,98.0,997.6,100.0,10.0,0.52,"Rain, Overcast","['72698599999', '72698024229']",437.0,North America,winter,90,0.0,0,Lo,Lo,Lo,Lo,SaLo,SaLo,0.545411,0.576050,0.606166,0.630767,0.652751
4,4,4,1996-12-26 00:00:00,Western Columbia River Gorge,The western part of the Columbia Gorge on the ...,Western Columbia River Gorge,50km,debris_flow,freeze_thaw,small,above_road,0.0,0.0,United States,Oregon,14095,Washougal,13.82547,-122.1828,45.5492,1.7,-0.7,0.1,1.7,-7.9,-3.5,-1.3,90.29,24.90,66.67,50.955865,37.1,97.9,997.6,100.0,9.9,0.52,"Rain, Overcast","['72698599999', '72698024229']",248.0,North America,winter,98,0.0,0,Lo,Lo,Lo,Lo,Lo,Lo,6.077988,6.419422,6.755037,7.029181,7.274169


In [2]:
def get_time_stamp(x):
    return int(time.mktime(datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S').timetuple()))
def get_date_time(x):
    return datetime.fromtimestamp(x).isoformat(" ")

In [207]:
# df = pd.read_csv("vn.csv", parse_dates= ["event_date"])

In [210]:
# x,y = train_test_split(df[["event_date","longitude","latitude"]], train_size = 0.8, test_size = 0.2, random_state = 0)

In [214]:
# x.to_csv("vn_train_timestamp.csv")
# y.to_csv("vn_test_timestamp.csv")

In [137]:
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
# # regressor=xgb.XGBRegressor(eval_metric='rmsle')
# regressor=xgb.XGBRegressor(eval_metric='rmse')
# #=========================================================================
# # exhaustively search for the optimal hyperparameters
# #=========================================================================
# from sklearn.model_selection import GridSearchCV
# # set up our search grid
# param_grid = {"max_depth":    [1, 2, 3, 4, 5, 6],
#               "n_estimators": [500, 600, 700],
#               "learning_rate": [0.001, 0.002]}

# # try out every combination of the above values
# search = GridSearchCV(regressor, param_grid, cv=5).fit(X_train, y_train)

# print("The best hyperparameters are ",search.best_params_)

# regressor=xgb.XGBRegressor(learning_rate = search.best_params_["learning_rate"],
#                            n_estimators  = search.best_params_["n_estimators"],
#                            max_depth     = search.best_params_["max_depth"],
#                            eval_metric='rmse')

# regressor.fit(X_train, y_train)
# y_pred = regressor.predict(X_test)
# rsme = np.sqrt(mean_squared_error(y_test,y_pred))
# rsme

(84,) (21,) (84,) (21,)
The best hyperparameters are  {'learning_rate': 0.002, 'max_depth': 6, 'n_estimators': 700}


1.4961557379214567

In [3]:
df_train = pd.read_csv("vn_train_timestamp.csv", parse_dates=["event_date"])
df_test = pd.read_csv("vn_test_timestamp.csv", parse_dates=["event_date"])


In [4]:
df_train["event_date"] = df_train["event_date"].apply(lambda x: get_time_stamp(x))
df_test["event_date"] = df_test["event_date"].apply(lambda x: get_time_stamp(x))

In [5]:
X_train_lon = df_train["event_date"]
y_train_lon = df_train["longitude"]
X_test_lon = df_test["event_date"]
y_test_lon = df_test["longitude"]

In [6]:
regressor_lon= xgb.XGBRegressor( max_depth     = 3,
                           n_estimators  = 500,
                           learning_rate = 0.01,
                           eval_metric='rmse')

regressor_lon.fit(X_train_lon, y_train_lon)
y_pred_lon = regressor_lon.predict(X_test_lon)
rsme_lon = np.sqrt(mean_squared_error(y_test_lon,y_pred_lon))
rsme_lon

1.3751238716858292

In [7]:
X_train_lat = df_train["event_date"]
y_train_lat = df_train["latitude"]
X_test_lat = df_test["event_date"]
y_test_lat = df_test["latitude"]

In [8]:
regressor_lat=xgb.XGBRegressor( max_depth = 3,
                           n_estimators  = 500,
                           learning_rate = 0.01,
                           eval_metric='rmse')

regressor_lat.fit(X_train_lat, y_train_lat)
y_pred_lat = regressor_lat.predict(X_test_lat)
rsme_lat = np.sqrt(mean_squared_error(y_test_lat,y_pred_lat))
rsme_lat

3.2895350794111957

In [23]:
result = pd.DataFrame(columns= ["time", "lon", "lat", "lon_pred", "lat_pred"])

In [24]:
result["time"] = X_test_lat.apply(lambda x: get_date_time(x))
# result["time"] = result["time"].apply(lambda x: get_date_time(x))
result["lon"] = y_test_lon
result["lat"] = y_test_lat
result["lon_pred"] = y_pred_lon
result["lat_pred"] = y_pred_lat

In [25]:
result

,time,lon,lat,lon_pred,lat_pred
0,2009-09-29 00:00:00,107.858400,15.598000,107.682732,16.691109
1,2012-08-31 01:00:00,106.693900,10.676000,107.437935,16.713028
2,2007-10-05 00:00:00,103.900000,21.316700,106.059715,19.790966
3,2012-09-27 00:00:00,104.001400,22.453900,107.437935,16.713028
4,2015-08-15 00:00:00,105.584500,20.893800,105.786682,21.109739
5,2010-11-15 00:00:00,108.659100,15.330700,108.235191,15.862576
6,2008-10-11 00:00:00,108.216700,15.330000,105.502731,20.651020
7,2017-06-28 13:10:00,105.050925,8.774708,105.401382,10.966474
8,2011-06-26 00:00:00,105.313100,19.943600,105.263000,18.893505
9,2010-11-11 00:00:00,108.710300,15.207400,108.235191,15.862576


In [26]:
result.to_csv("pred.csv")

###phase2

In [27]:
df = pd.read_csv("vn.csv")
df.head(5)

,Unnamed: 0,event_date,event_title,event_description,location_description,location_accuracy,landslide_category,landslide_trigger,landslide_size,landslide_setting,fatality_count,injury_count,country_name,admin_division_name,admin_division_population,gazeteer_closest_point,gazeteer_distance,longitude,latitude,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,precip,precipcover,windgust,windspeed,winddir,pressure,cloudcover,visibility,moonphase,conditions,stations,elevation,continent,season,treecover2000,loss,gain,soil_texture_0,soil_texture_10,soil_texture_30,soil_texture_60,soil_texture_100,soil_texture_200,population_density_2000,population_density_2005,population_density_2010,population_density_2015,population_density_2020
0,128,2007-07-01 00:00:00,"Cam Pha Town, Quang Ninh Province",140-257mm of rain; also landslides in Ha Long ...,"Cam Pha Town, Quang Ninh Province",5km,landslide,rain,medium,unknown,3.0,0.0,Vietnam,Quảng Ninh,135477,Cẩm Phả Mines,1.03388,107.2930,21.0233,27.7,24.9,26.2,33.1,24.9,27.5,25.7,96.82,23.6,37.50,50.955865,7.2,176.563614,1002.0,98.8,9.3,0.51,"Rain, Overcast",['48826099999'],250.0,Asia,summer,0,0.0,0,SaClLo,ClLo,ClLo,ClLo,ClLo,ClLo,424.017090,452.489838,479.269470,507.756012,530.224060
1,249,2007-10-05 00:00:00,Thanh Hoa,flooding and landslides affected entire provin...,Thanh Hoa,unknown,complex,tropical_cyclone,large,unknown,0.0,0.0,Vietnam,Thanh Hóa,0,Thị Trấn Ngọc Lặc,6.03262,105.3500,20.0400,27.9,24.4,25.7,33.4,24.4,27.5,24.9,95.39,134.6,37.50,50.955865,28.8,176.563614,1007.1,100.0,15.5,0.86,"Rain, Overcast",['48840099999'],164.0,Asia,autumn,55,0.0,0,ClLo,ClLo,ClLo,Cl,ClLo,ClLo,271.980103,269.272278,264.601135,260.073486,251.958771
2,248,2007-10-05 00:00:00,"Nghe An's Que Phong, Son La",multiple areas in North and Central Vietnam hi...,"Nghe An's Que Phong, Son La",25km,landslide,tropical_cyclone,large,unknown,0.0,0.0,Vietnam,Sơn La,19054,Sơn La,2.18698,103.9000,21.3167,24.1,21.4,22.3,24.1,21.4,22.3,21.4,94.73,64.0,37.50,50.955865,7.2,176.563614,1009.1,90.0,12.3,0.86,"Rain, Partially cloudy",['48806099999'],716.0,Asia,autumn,0,0.0,0,ClLo,ClLo,Cl,Cl,Cl,Cl,244.023453,269.444153,295.291870,323.696930,349.747528
3,260,2007-10-15 00:00:00,Quang Nam Province,"27 landslides in 6 mountainous districts, affe...",Quang Nam Province,unknown,landslide,rain,medium,unknown,0.0,0.0,Vietnam,Quảng Nam,0,Doug An,18.76169,107.9408,15.6045,26.1,24.1,24.7,26.1,24.1,24.7,23.6,93.74,35.3,13.04,50.955865,32.4,176.563614,1013.5,90.9,6.5,0.08,"Rain, Overcast",['48855099999'],29.0,Asia,autumn,99,0.0,0,ClLo,ClLo,ClLo,ClLo,ClLo,ClLo,66.296844,67.440483,68.091675,68.765678,68.450790
4,283,2007-10-30 00:00:00,Quang Nam,blocked roads in mountainous areas,Quang Nam,50km,landslide,tropical_cyclone,medium,unknown,3.0,0.0,Vietnam,Quảng Nam,0,Doug An,15.95575,107.9660,15.5946,28.1,23.1,25.4,32.8,23.1,25.7,23.6,89.74,70.4,16.67,50.955865,39.6,176.563614,1009.0,93.3,5.3,0.63,"Rain, Overcast",['48855099999'],129.0,Asia,autumn,87,0.0,0,ClLo,ClLo,ClLo,Cl,Cl,Cl,66.296852,67.440491,68.091682,68.765678,68.450790


In [87]:
def preprocessing(df_temp):
    dfs = df_temp.copy(deep = True)
    dfs.drop(['admin_division_population','population_density_2000', 'population_density_2005', 'population_density_2010', 'population_density_2015','tempmax','tempmin',"feelslikemax","feelslikemin"], axis="columns", inplace=True)
    ft = dfs.columns.difference(["landslide_size"])
    category_ft = dfs.select_dtypes(exclude=np.number).columns

    enc = OrdinalEncoder()
    enc.fit(dfs[category_ft])
    dfs[category_ft] = enc.transform(dfs[category_ft])

    labelencoder = LabelEncoder()
    dfs["landslide_size"] = labelencoder.fit_transform(dfs["landslide_size"])

    X = dfs[ft]
    y = dfs["landslide_size"]
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state = 0,stratify = y)
    
    # Scale features
    # scaler = MaxAbsScaler()
    # X_train = scaler.fit_transform(X_train)
    # X_test = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test

In [88]:
X_train, X_test, y_train, y_test = preprocessing(df)

In [95]:
xg = xgb.XGBClassifier(n_estimators = 10)
xg.fit(X_train,y_train)
xg_score=xg.score(X_test,y_test)
xg_y_predict=xg.predict(X_test)
y_true=y_test
xg_precision,xg_recall,xg_fscore,none= precision_recall_fscore_support(y_true, xg_y_predict, average='weighted')
print(xg_score,xg_precision,xg_recall,xg_fscore)

0.8095238095238095 0.655328798185941 0.8095238095238095 0.7243107769423559


c:\Users\tqdcr\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [98]:
joblib.dump(xg,"vn.sav")

['vn.sav']